In [8]:
import pandas as pd
import json
from google.cloud import storage
import os
import glob
import dat
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import re
from io import StringIO
import newlinejson as nlj
from json import loads, dumps

# import msgspec
import gcsfs
# import jsonlines

In [11]:
# os.environ['AZURE_STORAGE_ACCOUNT']

KeyError: 'AZURE_STORAGE_ACCOUNT'

In [3]:
def crate_new_bucket(bucket_name, project_id):
    # Instantiating storage class
    storage_client = storage.Client(project_id)
    # The name for the new bucket gathering from existing bucket
    bucket_name_conv = f"{bucket_name}_jsonl_conv" # + "jsonl_conv"
    # Creates the new bucket if bucket already exists using existing name with conversion
    if storage_client.bucket(bucket_name_conv).exists():
        bucket_new  = bucket_name_conv
    # If bucket does not exists crates it and gets its name
    else:
        bucket_new = storage_client.create_bucket(bucket_name_conv, location='europe-west8')
        bucket_new.location = 'europe-west8'
        print("created bucket {} in {}".format(bucket_new.name, bucket_new.location))
        bucket_new  = bucket_new.name
    # Returning created/existing bucket name as variable
    return bucket_new

def convert_json_jsonl(bucket_name,project_id):
    # Instantiating needed classes and methods
    storage_client = storage.Client(project_id)
    bucket = storage_client.bucket(bucket_name)
    blobs = storage_client.list_blobs(bucket_name)

    # New bucket creation
    new_bucket = crate_new_bucket(bucket_name,project_id)

    # Loop through blobs in bucket and converting nljson with saving into new bucket path
    for blob in blobs:
        # Getting json file content as string
        string_blob = blob.download_as_string(client=None)
        # Append all file contents into one string to get one table combining all on BQ
        string_comb = '\n'.join(string_blob)
    # Converting string to a dict JSON
    JSON_file = json.loads(string_comb)
    # Converting JSON to JSON New Line
    # nl_JSON_file = '\n'.join([json.dumps(JSON_file)])
    # That part is alternative if outer keys would like to be dispersed
    nl_JSON_file = '\n'.join([json.dumps(JSON_file[outer_key], sort_keys=True) 
                        for outer_key in sorted(JSON_file.keys(),
                                                key=lambda x: int(x))])
    # Calling new bucket name meta
    bucket_new = storage_client.bucket(new_bucket)
    # Blob name creation for converted blobs
    blob_name = f'{blob.name}_converted_jsonl'
    # Getting blob variable to apply read/write operations
    blob = bucket_new.blob(blob_name)
    # Writing new blob to new bucket
    with blob.open("w") as f:
        f.write(nl_JSON_file)
    return 'files_on_bucket_converted_toJSONnl'

In [4]:
convert_json_jsonl("tryoutdavar",'capable-memory-417812')

[2024-03-25T10:58:10.447+0000] {_default.py:683} WARNING - No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable
[2024-03-25T10:58:10.497+0000] {_default.py:683} WARNING - No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable


TypeError: sequence item 0: expected str instance, int found

In [65]:
def crate_new_bucket(bucket_name, project_id):
    # Instantiating storage class
    storage_client = storage.Client(project_id)
    # The name for the new bucket gathering from existing bucket
    bucket_name_conv = f"{bucket_name}_jsonl_conv" # + "jsonl_conv"
    # Creates the new bucket if bucket already exists using existing name with conversion
    if storage_client.bucket(bucket_name_conv).exists():
        bucket_new  = bucket_name_conv
    # If bucket does not exists crates it and gets its name
    else:
        bucket_new = storage_client.create_bucket(bucket_name_conv, location='europe-west8')
        bucket_new.location = 'europe-west8'
        print("created bucket {} in {}".format(bucket_new.name, bucket_new.location))
        bucket_new  = bucket_new.name
    # Returning created/existing bucket name as variable
    return bucket_new

def convert_json_jsonl(bucket_name,project_id):
    # Instantiating needed classes and methods
    storage_client = storage.Client(project_id)
    bucket = storage_client.bucket(bucket_name)
    blobs = storage_client.list_blobs(bucket_name)

    # New bucket creation
    new_bucket = crate_new_bucket(bucket_name,project_id)
    # Combine JSONS
    d_json = {}
    global blob_name
    blob_name = 'temp_blob'
    # Loop through blobs in bucket and converting nljson with saving into new bucket path
    for blob in blobs:
        # Getting json file content as string and converting to dict
        # JSON_file = json.loads(blob.download_as_string(client=None))
        d_json.update(json.loads(blob.download_as_string(client=None)))
        # Blob name creation for converted blobs
        # blob_name = f'{blob.name}_converted_jsonl'
        blob_name = blob_name = 'epl_2022_2023_season_stats.json'
    # Converting to JSON to JSON New Line
    # nl_JSON_file = '\n'.join([json.dumps(d_json)])
    # That part is alternative if outer keys would like to be dispersed
    nl_JSON_file = '\n'.join([json.dumps(d_json[outer_key], sort_keys=True) 
                        for outer_key in sorted(d_json.keys(),
                                                key=lambda x: int(x))])
    # Calling new bucket name meta
    bucket_new = storage_client.bucket(new_bucket)
    # Getting blob variable to apply read/write operations
    blob = bucket_new.blob(blob_name)
    # Writing new blob to new bucket
    with blob.open("w") as f:
        f.write(nl_JSON_file)
    return 'files_on_bucket_converted_toJSONnl'

In [66]:
convert_json_jsonl("tryoutdavar",'capable-memory-417812')

[2024-03-25T12:36:16.823+0000] {_default.py:683} WARNING - No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable
[2024-03-25T12:36:16.838+0000] {_default.py:683} WARNING - No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable


'files_on_bucket_converted_toJSONnl'

In [114]:
def write_to_bq(project_id, bucket_name):
    # Instantiating big query client
    bq_client = bigquery.Client(project_id)
    # Getting bucket variable as source
    bucket = storage.Client().bucket(bucket_name)
    # If there are folders in bucket prefix part could be used
    for blob in bucket.list_blobs(prefix=""):
        # Checking for json blobs as list_blobs also returns folder_name
        if ".json" in blob.name: 
            # Parsing JSON file name to create table on BQ with same file name
            json_filename = blob.name.split('.json')[0]
        #    json_filename = re.findall(r".*/(.*).json",blob.name) #Extracting file name for BQ's table id
            # Determining table name
            table_id = f"capable-memory-417812.premiership.{json_filename}" 
            try: # Check if table exists and operate TRUNCATE/WRITE ROWS
               bq_client.get_table(table_id)
               uri = f"gs://{bucket_name}/{blob.name}"
               # Creating configs for load job operations (append)
               job_config = bigquery.LoadJobConfig(
               autodetect=True,
               write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
               source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON)
               print("Table {} already exists. Rows append operation be done.".format(table_id))
               load_job = bq_client.load_table_from_uri(
                     uri, table_id, location = 'europe-west8',  
                     job_config=job_config,
               )  
               # Make an API request.
               load_job.result()  
               # Waits for the job to complete.
               destination_table = bq_client.get_table(table_id)  
               # Make an API request.
               print("Total {} rows exits on the table.".format(destination_table.num_rows))
           # If table is not found, upload it WRITE If table empty.  
            except NotFound:   
               uri = f"gs://{bucket_name}/{blob.name}"
               print(uri)
               job_config = bigquery.LoadJobConfig(
               autodetect=True,
               write_disposition=bigquery.WriteDisposition.WRITE_EMPTY,
               source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON)
               load_job = bq_client.load_table_from_uri(
                     uri, table_id, location = 'europe-west8',  
                     job_config=job_config,
               )  
               # Make an API request.
               load_job.result()  
               # Waits for the job to complete.
               destination_table = bq_client.get_table(table_id)  # Make an API request.
               print("Loaded {} rows.".format(destination_table.num_rows))

In [115]:
write_to_bq('capable-memory-417812',"tryoutdavar_jsonl_conv")

Table capable-memory-417812.premiership.epl_2022_2023_07_02_2023 already exists. Rows append operation be done.
Total 1 rows exits on the table.
Table capable-memory-417812.premiership.epl_2022_2023_07_02_2024 already exists. Rows append operation be done.
Total 1 rows exits on the table.


In [10]:
# df = pd.read_json('gs://game_data_giray/epl_2022_2023_07_02_2023.json')
# df = pd.read_json('gs://game_data_giray/premiership_short.json')

In [11]:
def list_blob_elements(bucket_name):
    """Lists blob elements in mentioned bucket"""

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blobs = storage_client.list_blobs(bucket_name)
    # blob = bucket.blob(blob_name)
    list_blobs = []
   
    for blob in blobs:
        list_blobs.append(blob.name)
        
    return list_blobs

In [13]:
list_blob_elements("game_data_giray_jsonl_conv")

['epl_2022_2023_07_02_2023.json_nl.json',
 'epl_2022_2023_07_02_2024.json_nl.json']

In [2]:
def crate_new_bucket(bucket_name, project_id):
    # Instantiating storage class
    storage_client = storage.Client(project_id)
    # The name for the new bucket gathering from existing bucket
    bucket_name_conv = f"{bucket_name}_jsonl_conv" # + "jsonl_conv"
    print(bucket_name_conv)
    # Creates the new bucket if bucket already exists using existing name with conversion
    if storage_client.bucket(bucket_name_conv).exists():
        print('eben')
        bucket_new  = bucket_name_conv
    # If bucket does not exists crates it and gets its name
    else:
        bucket_new = storage_client.create_bucket(bucket_name_conv, location='europe-west8')
        bucket_new.location = 'europe-west8'
        print("created bucket {} in {}".format(bucket_new.name, bucket_new.location))
        bucket_new  = bucket_new.name
    return bucket_new


def convert_json_jsonl(bucket_name, project_id, blob_name = [], path = []):
    ''' This function reads multiple json files from location passed as parameter 
    and converts them to jsonl and saves into created bucket with an inner function
    if bucket exists uses existing bucket'''
    # New bucket creation
    new_bucket = crate_new_bucket(bucket_name,project_id)

    # Instantiating needed classes and methods
    storage_client = storage.Client(project_id)
    bucket = storage_client.bucket(bucket_name)
    blobs = storage_client.list_blobs(bucket_name)
    
    # Loop through blobs in bucket and converting nljson with saving into new bucket path
    for blob in blobs:
        print(blob.name)
        df = pd.read_json("gs://{}/{}".format(bucket_name, blob.name)) 
        df.to_json(f'gs://{new_bucket}/{blob.name}',orient="records",lines=True)
    return 'json_files_converted_nljson'

In [3]:
convert_json_jsonl("game_data_giray",'capable-memory-417812')

game_data_giray_jsonl_conv


/tmp/ipykernel_9412/1572037636.py:14: DeprecationWarning: Assignment to 'Bucket.location' is deprecated, as it is only valid before the bucket is created. Instead, pass the location to `Bucket.create`.
  bucket_new.location = 'europe-west8'


created bucket game_data_giray_jsonl_conv in europe-west8
epl_2022_2023_07_02_2023.json
epl_2022_2023_07_02_2024.json


'json_files_converted_nljson'